In [2]:
using Pkg; Pkg.activate(".")
using HypergraphModularity

# using Optim 
using Plots
using TimerOutputs
using StatsBase

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [190]:
kmax = 3
n = 100
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

α0 = vcat([3.0, 3.0, 10.0], [0.2, 0.7, 0.5])

function ω(p, α)
    k = sum(p)
    return sum(p)/sum((p .* (1:length(p)).^α[k])) / n^(α[kmax+k]*k)
end

function ω₃(p, α)
    k = sum(p)
    return (length(p))^(-α[k])/n^(α[kmax+k]*k)
end

Ω₁ = buildΩ(ω, α0, kmax)
Ω₃ = buildΩ(ω₃, α0, kmax)

# all-or-nothing cut with dyadic edges always between group
function Ω₂(z; α, mode = "group")
    k = (mode == "group") ? length(z) : sum(z)
    
    if k <= 2
        if mode == "group"
            if length(unique!(z)) == 1
                return (10^(-3))^(α[k])
            else 
                return length(unique!(z))^(-α[k])
            end
        elseif mode == "partition"
            if length(z) == 1
                return (10^(-3))^(α[k])
            else 
                return length(z)^(-α[k])
            end
        end
    else
        if mode == "group"
            return length(unique!(z))^(-α[k]) / n^(α[kmax+k]*k)
        elseif mode == "partition"
            return length(z)^(-α[k]) / n^(α[kmax+k]*k)
        end
    end
end

Ω = Ω₂

H = sampleSBM(Z, ϑ, Ω;α=α0, kmax=kmax, kmin = 1)
H.E[1] = Dict();

In [191]:
for k = 2:kmax
    p = mean([length(partitionize(Z[e])) == 1 for e in keys(H.E[k])])
    println("k = $k: $(round(100*p, digits = 0)) % of $(length(keys(H.E[k]))) edges are within a single group.")
end

k = 2: 0.0 % of 1072 edges are within a single group.
k = 3: 100.0 % of 844 edges are within a single group.


In [199]:
(844*2 - 1072)/n, sqrt(2*((844*2 + 1072)/n))  

(6.16, 7.429670248402684)

# Dyadic Modularity

In [192]:
γ̂ = .9
α̂ = α0

timeAlg(expr)= @timed eval(expr)

println("DYADIC")

print(rpad("iteration", 12))
print(rpad("Q (dyadic)", 15))
print(rpad("L (dyadic)", 20))
print(rpad("Q (polyadic)", 15))
print(rpad("γ̂", 15))
print(rpad("groups", 10))
println(rpad("time (s)", 10))
println(rpad("",  100, "-"))

weighted = false

Ẑ_dyadic = copy(Z)

print(rpad("true", 12))
γ̂ = computeDyadicResolutionParameter(H, Z)
Q = dyadicModularity(H, Z, γ̂;weighted=weighted)
α̂ = coordinateAscent(H, Z, Ω, α̂; n_iters = 10, amin = -10, amax = 10)
ω_in, ω_out = computeDyadicResolutionParameter(H, Z; mode="ω", weighted=weighted)
L_D = dyadicLogLikelihood(H, Z, ω_in, ω_out, weighted=weighted)

Q_H = modularity(H, Z, Ω; α = α̂)

print(rpad("$(round(Q_D, digits = 3))", 15))
print(rpad("$(round(L_D, digits = 3))", 20))
print(rpad("$(round(Q_H, digits = 0))", 15))
print(rpad("$(round(γ̂, digits = 3))", 15))
println(rpad("$(length(unique(Ẑ_dyadic)))", 10))

for i = 1:20
    out = timeAlg(:(CliqueExpansionModularity(H, γ̂, weighted, true))) # second true adds random scan order
    Ẑ_dyadic = out[1]
    time = out[2]
    
    γ̂ = computeDyadicResolutionParameter(H, Ẑ_dyadic)
    α̂ = coordinateAscent(H, Ẑ_dyadic, Ω, α̂; n_iters = 10, amin = -10, amax = 10)
    
    Q_D = dyadicModularity(H, Ẑ_dyadic, γ̂;weighted=weighted)
    
    ω_in, ω_out = computeDyadicResolutionParameter(H, Ẑ_dyadic; mode="ω", weighted=weighted)
    L_D = dyadicLogLikelihood(H, Ẑ_dyadic, ω_in, ω_out, weighted=weighted)
    
    Q_H = modularity(H, Ẑ_dyadic, Ω; α = α̂)
    
    print(rpad("$i", 12))
    print(rpad("$(round(Q_D, digits = 3))", 15))
    print(rpad("$(round(L_D, digits = 3))", 20))
    print(rpad("$(round(Q_H, digits = 0))", 15))
    print(rpad("$(round(γ̂, digits = 3))", 15))
    print(rpad("$(length(unique(Ẑ_dyadic)))", 10))
    println(rpad("$(round(time; digits=3))", 10))
    
end

DYADIC
iteration   Q (dyadic)     L (dyadic)          Q (polyadic)   γ̂             groups    time (s)  
----------------------------------------------------------------------------------------------------
true        0.109          -6.326584963e7      -36389.0       0.998          2         
1           0.132          -6.3265838187e7     -40833.0       0.999          2         0.008     
2           0.127          -6.3265858815e7     -47385.0       1.0            2         0.008     
3           0.122          -6.3265840707e7     -46574.0       1.017          4         0.008     
4           0.113          -6.3265865496e7     -49582.0       1.022          4         0.008     
5           0.124          -6.3265839072e7     -45965.0       1.012          4         0.01      
6           0.134          -6.3265827361e7     -45471.0       1.0            2         0.008     
7           0.13           -6.3265847387e7     -45560.0       0.999          3         0.022     
8           0.128   

# Polyadic Modularity

In [193]:
α̂ = α0

println("POLYADIC")

print(rpad("iteration", 20))
print(rpad("Q", 15))
print(rpad("groups", 10))
println(rpad("time (s)", 10))
println(rpad("",  65, "-"))

Ẑ = zero(Z)

for i = 1:20
    out = timeAlg(:(SuperNodeLouvain(H,kmax,Ω;α=α̂, verbose=false, scan_order ="random")))
    Ẑ = out[1]
    time = out[2]
    
    
    α̂ = coordinateAscent(H, Ẑ, Ω, α̂; n_iters = 10, amin = -10, amax = 10)
    Q = modularity(H, Ẑ, Ω; α = α̂)
    
    print(rpad("$i", 20))
    print(rpad("$(round(Q, digits = 0))", 15))
    print(rpad("$(length(unique(Ẑ)))", 10))
    println(rpad("$(round(time; digits=3))", 10))
end

POLYADIC
iteration           Q              groups    time (s)  
-----------------------------------------------------------------
1                   -48105.0       86        1.938     
2                   -47309.0       32        1.244     
3                   -44184.0       6         0.476     
4                   -36389.0       2         0.24      
5                   -36389.0       2         0.24      
6                   -36389.0       2         0.248     
7                   -36389.0       2         0.233     
8                   -36389.0       2         0.249     
9                   -36389.0       2         0.216     
10                  -36389.0       2         0.226     
11                  -36389.0       2         0.256     
12                  -36389.0       2         0.218     
13                  -36389.0       2         0.226     
14                  -36389.0       2         0.228     
15                  -36389.0       2         0.223     
16                  -36389.0 

# Figures (trying RCall)

In [27]:
using RCall
RCall.rcall_p(:options, rcalljl_options=Dict(:width => 600, :height => 300))
R"""
library(tidyverse)
"""

┌ Warning: RCall.jl: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──
│ ✔ ggplot2 3.3.2     ✔ purrr   0.3.4
│ ✔ tibble  3.0.3     ✔ dplyr   1.0.0
│ ✔ tidyr   1.1.0     ✔ stringr 1.4.0
│ ✔ readr   1.3.1     ✔ forcats 0.5.0
│ ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
│ ✖ dplyr::filter() masks stats::filter()
│ ✖ dplyr::lag()    masks stats::lag()
└ @ RCall /home/phil/.julia/packages/RCall/Qzssx/src/io.jl:160


RObject{StrSxp}
 [1] "forcats"   "stringr"   "dplyr"     "purrr"     "readr"     "tidyr"    
 [7] "tibble"    "ggplot2"   "tidyverse" "stats"     "graphics"  "grDevices"
[13] "utils"     "datasets"  "methods"   "base"     
